<h3>Logistic Regression on MNIST dataset using Tensorflow.</h3>
<br>

In [4]:
import tensorflow as tf
import numpy as np
from __future__ import absolute_import, division, print_function

In [5]:
num_classes = 10 

num_features = 784

learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50

In [9]:
from tensorflow.keras.datasets import mnist

(inputs_train, targets_train), (inputs_test, targets_test) = mnist.load_data()

inputs_train, inputs_test = np.array(inputs_train, np.float32), np.array(inputs_test, np.float32)

inputs_train, inputs_test = inputs_train.reshape([-1, num_features]), inputs_test.reshape([-1, num_features])

inputs_train, inputs_test = inputs_train / 255, inputs_test / 255

In [11]:
train_data = tf.data.Dataset.from_tensor_slices((inputs_train, targets_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [12]:
W = tf.Variable(tf.ones([num_features, num_classes]), name = "weight")

b = tf.Variable(tf.zeros([num_classes]), name = "bias")

In [13]:
def logistic_regression(x):

    return tf.nn.softmax(tf.matmul(x, W) + b)

def cross_entropy(y_pred, y_true):

    y_true = tf.one_hot(y_true, depth=num_classes)
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)

    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

In [14]:
def accuracy(y_pred, y_true):
  
  correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
  
  return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

optimizer = tf.optimizers.SGD(learning_rate)

In [15]:
def run_optimization(x, y):

    with tf.GradientTape() as g:
        
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)

    gradients = g.gradient(loss, [W, b])
    optimizer.apply_gradients(zip(gradients, [W, b]))

In [16]:
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps), 1):
    
    run_optimization(batch_x, batch_y)

    if step % display_step == 0:

        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)

        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))

step: 50, loss: 528.999451, accuracy: 0.839844
step: 100, loss: 108.224426, accuracy: 0.894531
step: 150, loss: 305.229431, accuracy: 0.730469
step: 200, loss: 135.933777, accuracy: 0.890625
step: 250, loss: 80.160927, accuracy: 0.921875
step: 300, loss: 324.033691, accuracy: 0.746094
step: 350, loss: 71.844398, accuracy: 0.917969
step: 400, loss: 192.744522, accuracy: 0.820312
step: 450, loss: 58.223103, accuracy: 0.921875
step: 500, loss: 81.265228, accuracy: 0.910156
step: 550, loss: 100.616714, accuracy: 0.851562
step: 600, loss: 142.453857, accuracy: 0.851562
step: 650, loss: 72.150513, accuracy: 0.906250
step: 700, loss: 70.402649, accuracy: 0.925781
step: 750, loss: 165.765366, accuracy: 0.871094
step: 800, loss: 41.746349, accuracy: 0.941406
step: 850, loss: 39.642990, accuracy: 0.941406
step: 900, loss: 183.746918, accuracy: 0.855469
step: 950, loss: 234.977417, accuracy: 0.847656
step: 1000, loss: 40.009808, accuracy: 0.945312


In [17]:
pred = logistic_regression(inputs_test)

print("Test Accuracy: %f" % accuracy(pred, targets_test))

Test Accuracy: 0.913900
